In [1]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import h5py
import hdf5plugin
import scipy as sp

#plt.figure(dpi=600)

In [5]:
# Define a data reader
class DataReader:
    
    def __init__(self, data_dir, filename, metadata_file_name):
        self.filename = filename
        self.prefix = filename.replace('.h5','')
        self.file_path = os.path.join(data_dir, filename)
        self.md = pd.read_csv(metadata_file_name)
    
    def get_query_metadata(self, **kwargs):
        """
        Filters metadata according to variable query args (kwargs).
        """
        qmd = self.md.copy()
        for arg in kwargs:
            qmd = qmd.loc[qmd[arg] == kwargs[arg]].copy()
            
        return qmd
    
    def get_nb_of_rows(self):
        f = h5py.File(self.file_path,'r')
        nrows = f[self.filename.replace('.h5','')]['axis1'].shape[0]
        return nrows
    
    def query_data(self, **kwargs):
        """
        This function uses h5py to query the file. It returns a dataframe (which might well be huge). Depending on your query,
        you might still run out of RAM and your session crash. Call it like:
        
        Examples:
        query_data(day = day_nb, donor = donor_nb, cell_type = some_type_you_want)
        query_data(day = day_nb, cell_type = some_type_you_want)
        query_data(day = day_nb) <-- Beware: this might return lots of data that do not fit into RAM, especially for multiome files.
        query_data(donor = donor_nb) <-- Same remark holds.
        """
        
        f = h5py.File(self.file_path, 'r')
        query_df = self.get_query_metadata(**kwargs)

        ax1 = f[self.prefix]['axis1'][:]
        ax1 = list(map(lambda x: x.decode(), ax1))
        ax1 = pd.DataFrame(ax1, columns = ['cell_id']).reset_index().\
        rename(columns = {'index':'index_col'})
        ax1 = pd.merge(query_df[['cell_id','day','donor','cell_type']]
                      ,ax1
                      ,on = 'cell_id').sort_values(by = 'index_col')
        #ax1 = ax1.iloc[0:50,:]
        
        out = f[self.prefix]['block0_values'][ax1.index_col.values,:]
        columns = [col.decode() for col in f[self.prefix]['axis0'][:]]
        out = pd.DataFrame(out, index = ax1.cell_id, columns = columns)
        
        f.close()

        return out, ax1
    

In [6]:
import time

def h52h5ad(input_file, sparse=False, chosen_for_train=True):
    dr_atac = DataReader('./raw/', input_file, 'raw/metadata.csv')
    start = time.time()
    mtx_tmp, meta_tmp = dr_atac.query_data(chosen_for_train=chosen_for_train)
    print('h5 query extraction done! %.2fs past.'%(time.time()-start))
    
    meta_tmp.index = meta_tmp['cell_id']
    
    anndat_out = ad.AnnData(
        X = mtx_tmp, #.to_numpy()
        obs = meta_tmp,
    )
    
    if sparse:
        anndat_out.X = sp.sparse.csc_matrix(anndat_out.X)
    
    return anndat_out


In [7]:
# H5 file saved in ./raw/ together with metadata.csv
multiome_atac_file = 'train_multi_inputs.h5'
multiome_rna_file = 'train_multi_targets.h5'

In [8]:
anndat_rnam = h52h5ad(multiome_rna_file)
anndat_rnam.write_h5ad('h5ad/HSPC_Multiome_RNA.h5ad')

h5 query extraction done! 111.57s past.


In [9]:
anndat_atac = h52h5ad(multiome_atac_file)
anndat_atac.write_h5ad('h5ad/HSPC_Multiome_ATAC.h5ad')

h5 query extraction done! 622.48s past.
